---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevin-crotty.com)
    
Jones Graduate School of Business, Rice University

---


# EXAMPLE DATA

In [10]:
K = 50          # strike price
T = 1           # years to maturity
sigma = 0.4     # volatility of underlying
r = 0.02        # risk-free rate
q = 0.03        # dividend yield

# FUNCTIONS

In [11]:
import numpy as np
import pandas as pd
from scipy.stats import norm


def callBS(S, K, T, sigma, r, q):
    def f(s):
        s = s if s != 0 else 1.0e-6
        d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        return np.exp(-q * T) * s * norm.cdf(d1) - np.exp(-r * T) * K * norm.cdf(d2)

    if isinstance(S, list) or isinstance(S, np.ndarray):
        return np.array([f(s) for s in S])
    else:
        return f(S)


def putBS(S, K, T, sigma, r, q):
    def f(s):
        s = s if s != 0 else 1.0e-6
        d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        return np.exp(-r * T) * K * norm.cdf(-d2) - np.exp(-q * T) * s * norm.cdf(-d1)

    if isinstance(S, list) or isinstance(S, np.ndarray):
        return np.array([f(s) for s in S])
    else:
        return f(S)


def callGreeks(S, K, T, sigma, r, q):
    def f(s):
        s = s if s != 0 else 1.0e-6
        d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        # delta
        delta = np.exp(-q * T) * norm.cdf(d1)
        # gamma
        gamma = np.exp(-q * T) * (1 / (s * sigma * np.sqrt(T))) * norm.pdf(d1)
        # theta
        theta = -np.exp(-q * T) * (s * sigma / (2 * np.sqrt(T))) * norm.pdf(d1)
        theta += q * np.exp(-q * T) * s * norm.cdf(d1)
        theta += -r * np.exp(-r * T) * K * norm.cdf(d2)
        # vega
        vega = np.exp(-q * T) * s * norm.pdf(d1) * np.sqrt(T)
        # rho
        rho = T * np.exp(-r * T) * K * norm.cdf(d2)
        return delta, gamma, theta, vega, rho

    if isinstance(S, list) or isinstance(S, np.ndarray):
        return np.array([f(s) for s in S])
    else:
        return f(S)


def putGreeks(S, K, T, sigma, r, q):
    def f(s):
        s = s if s != 0 else 1.0e-6
        d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        # delta
        delta = np.exp(-q * T) * norm.cdf(d1)
        delta += -np.exp(-q * T)
        # gamma
        gamma = np.exp(-q * T) * (1 / (s * sigma * np.sqrt(T))) * norm.pdf(d1)
        # theta
        theta = -np.exp(-q * T) * (s * sigma / (2 * np.sqrt(T))) * norm.pdf(d1)
        theta += q * np.exp(-q * T) * s * norm.cdf(d1)
        theta += -r * np.exp(-r * T) * K * norm.cdf(d2)
        theta += r * np.exp(-r * T) * K - q * np.exp(-q * T) * s
        # vega
        vega = np.exp(-q * T) * s * norm.pdf(d1) * np.sqrt(T)
        # rho
        rho = T * np.exp(-r * T) * K * norm.cdf(d2)
        rho += -T * np.exp(-r * T) * K
        return delta, gamma, theta, vega, rho

    if isinstance(S, list) or isinstance(S, np.ndarray):
        return np.array([f(s) for s in S])
    else:
        return f(S)

# CALCULATIONS

In [12]:
S = np.linspace(1, 100, 100)
callgreeks = callGreeks(S, K, T, sigma, r, q)
putgreeks  = putGreeks(S, K, T, sigma, r, q)

# FIGURES

In [13]:
import plotly.graph_objects as go

fig_delta = go.Figure()
fig_gamma = go.Figure()
fig_theta = go.Figure()
fig_vega  = go.Figure()
fig_rho   = go.Figure()

for i, fig, fig_name in zip(
    range(5),
    [fig_delta, fig_gamma, fig_theta, fig_vega, fig_rho],
    ["Delta", "Gamma", "Theta", "Vega", "Rho"],
):
    fig.add_trace(
        go.Scatter(x=S, y=callgreeks[:, i], mode="lines", name="European Call")
    )
    fig.add_trace(
        go.Scatter(x=S, y=putgreeks[:, i], mode="lines", name="European Put")
    )

    fig.update_layout(
        xaxis_title="Underlying Price",
        xaxis_tickformat=",.0f",
        yaxis_tickformat=",.2f",
        hovermode="x unified",
        template="plotly_white",
        title=dict(
            text=fig_name,
            yanchor="bottom",
            y=0.96,
            xanchor="center",
            x=0.2
        ),
        legend=dict(
            yanchor="top", 
            y=0.99, 
            xanchor="left", 
            x=0.01
        )
    )

In [14]:
# Delta
fig_delta.show()

In [15]:
# Vega
fig_vega.show()

In [16]:
# Gamma
fig_gamma.show()

In [17]:
# Theta
fig_theta.show()

In [18]:
# Rho
fig_rho.show()